# 🧠 Modélisation des prix au m² – Ville de Lille

Dans ce notebook, nous entraînons et comparons plusieurs modèles de machine learning pour prédire le **prix au m²** de logements (appartements et maisons) à Lille, à partir des données filtrées issues de la base DVF 2022.

L'objectif est d'identifier le modèle offrant la **meilleure précision**, puis de le conserver pour l’API de prédiction finale.

## 📦 Importation des bibliothèques

On commence par importer toutes les bibliothèques nécessaires pour :
- la manipulation de données (`pandas`, `numpy`),
- la visualisation (`matplotlib`, `seaborn`),
- les modèles de machine learning (`scikit-learn`, `xgboost`),
- les métriques d’évaluation,
- la sérialisation des modèles (`joblib`).



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import seaborn as sns
import joblib

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

## 📥 Chargement des données

On charge les données nettoyées de Lille depuis un fichier `.parquet`, puis on filtre pour ne garder que les biens immobiliers avec **exactement 4 pièces**. Cela permet de standardiser les observations pour une comparaison plus fiable.


In [ ]:
# Chargement depuis parquet
df = pd.read_parquet("../data/clean/lille_2022.parquet")

# Filtrer biens avec exactement 4 pièces principales
df_4p = df[df['Nombre pieces principales'] == 4].copy()

## 📊 Sélection des colonnes

On sélectionne les colonnes pertinentes pour l'entraînement du modèle, notamment :
- la surface bâtie,
- le nombre de lots,
- la surface de terrain,
- la valeur foncière,
- le type de logement (appartement ou maison),
- le prix au m² (notre variable cible).

In [ ]:
colonnes_a_garder = [
    'Surface reelle bati',
    'Nombre pieces principales',
    'Nombre de lots',
    'Valeur fonciere',
    'Surface terrain',
    'Code type local',
    'prix_m2'
]

df_logements = df_4p[colonnes_a_garder].copy()

## 🔍 Inspection des types de données

On vérifie les types de colonnes pour s'assurer qu’il n’y a pas d’incohérences avant de procéder aux transformations.

In [ ]:
df_logements.dtypes

## 🧼 Nettoyage des valeurs manquantes

- Les surfaces de terrain sont remplacées par `0` si elles sont manquantes.
- Cela est logique car certains logements, comme des appartements, peuvent ne pas avoir de terrain associé.
- On supprime ensuite les lignes restantes avec des valeurs nulles et les doublons.

In [ ]:
# Remplacer Surface terrain NaN par 0 pour les appartements (Code type local == 2)
df_logements.loc[
    (df_logements["Code type local"] == 2) & (df_logements["Surface terrain"].isna()),
    "Surface terrain"
] = 0

# Remplacer Surface terrain NaN par 0 pour les maisons (Code type local == 1)
df_logements.loc[
    (df_logements["Code type local"] == 1) & (df_logements["Surface terrain"].isna()),
    "Surface terrain"
] = 0

df_logements = df_logements.dropna()
df_logements = df_logements.drop_duplicates()

print(len(df_logements))

## 🏠 Séparation entre maisons et appartements

On sépare les données selon le type de bien (`Code type local`) :
- `1` correspond aux maisons,
- `2` aux appartements.

Cela permet de créer des modèles spécifiques à chaque type de bien.

In [ ]:
#séparation du dataset
df_maison = df_logements[df_logements["Code type local"] == 1]
df_appartement = df_logements[df_logements["Code type local"] == 2]

## 🚨 Détection des outliers

Fonction pour détecter les valeurs aberrantes (outliers) à l’aide de l’IQR (interquartile range) :
- Les valeurs situées à plus de 3 fois l’IQR au-dessus ou en dessous de Q1/Q3 sont considérées comme extrêmes.
- Cela permet de repérer les anomalies dans des colonnes comme `prix_m2`.

In [ ]:
def detect_outliers(df, column):
    
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return lower_bound, upper_bound, outliers

## 🔍 Visualisation des valeurs aberrantes détectées

On affiche les valeurs extrêmes détectées dans la colonne `prix_m2` pour les maisons et les appartements. Ces valeurs seront ensuite corrigées ou supprimées pour éviter de fausser l’apprentissage des modèles.

In [ ]:
# Pour df_logements
print("📦 Outliers prix_m2 (maisons) :")
print(detect_outliers(df_maison, "prix_m2")[2], "\n")

print("📦 Outliers prix_m2 (appartements) :")
print(detect_outliers(df_appartement, "prix_m2")[2], "\n")

## 🧽 Nettoyage des outliers

Deux fonctions sont utilisées :
- `remove_outliers()` : supprime directement les lignes avec valeurs extrêmes.
- `median_outliers()` : remplace les valeurs aberrantes par la **médiane** de la colonne concernée, afin de conserver plus d'observations sans biais extrême.

On applique ici `median_outliers` à toutes les colonnes importantes :
- `prix_m2`
- `Nombre de lots`
- `Surface terrain`
- `Surface reelle bati`

In [ ]:
def remove_outliers(df, column):
    lower, upper, _ = detect_outliers(df, column)
    return df[(df[column] >= lower) & (df[column] <= upper)]

def median_outliers(df, column):
    lower, upper, _ = detect_outliers(df, column)
    mediane = df[column].median()
    df.loc[(df[column] < lower) | (df[column] > upper), column] = mediane
    return df

## ✅ Résultat après nettoyage

On affiche le nombre de lignes restantes pour chaque dataset (`maison` et `appartement`) après le remplacement des outliers. Cela permet de vérifier qu’aucune donnée n’a été supprimée.

In [ ]:
df_maison_clean = median_outliers(df_maison, "prix_m2")
df_appartement_clean = median_outliers(df_appartement, "prix_m2")

df_maison_clean = median_outliers(df_maison, "Nombre de lots")
df_appartement_clean = median_outliers(df_appartement, "Nombre de lots")

df_maison_clean = median_outliers(df_maison, "Surface terrain")
df_appartement_clean = median_outliers(df_appartement, "Surface terrain")

df_maison_clean = median_outliers(df_maison, "Surface reelle bati")
df_appartement_clean = median_outliers(df_appartement, "Surface reelle bati")


print(len(df_maison_clean))
print(len(df_appartement_clean))

## 📊 Visualisation de la distribution du prix au m²
Cette section permet de visualiser la répartition des prix au m² pour les maisons et les appartements à Lille, avant tout entraînement de modèle. Cela permet d’identifier les tendances globales et de détecter visuellement d’éventuelles valeurs extrêmes.

In [ ]:
# Créer la figure et les 2 sous-graphiques côte à côte
fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# --- 1. Histogramme pour les maisons ---
axs[0].hist(df_maison_clean['prix_m2'], bins=30, color='orange', edgecolor='w')
axs[0].axvline(np.median(df_maison_clean['prix_m2']), color='red', linestyle='--', label='Médiane')
axs[0].set_title("Maisons - prix_m2")
axs[0].set_xlabel("prix_m2")
axs[0].set_ylabel("Nombre de biens")
axs[0].grid(True)
axs[0].legend()

# --- 2. Histogramme pour les appartements ---
axs[1].hist(df_appartement_clean['prix_m2'], bins=30, color='orange', edgecolor='w')
axs[1].axvline(np.median(df_appartement_clean['prix_m2']), color='red', linestyle='--', label='Médiane')
axs[1].set_title("Appartements - prix_m2")
axs[1].set_xlabel("prix_m2")
axs[1].grid(True)
axs[1].legend()

# Ajuster l'espacement
plt.tight_layout()
plt.show()

## 🧼 Nettoyage complémentaire des extrêmes de prix_m2

Afin d’améliorer la qualité des données d'entraînement, un filtrage manuel des outliers a été appliqué sur le prix au m². L’objectif est d’écarter les valeurs aberrantes susceptibles de fausser l’apprentissage des modèles.

### 🎯 Règles appliquées :
- **Maisons** : uniquement les biens dont le prix au m² est **compris entre 1 500 € et 5 000 €**
- **Appartements** : uniquement les biens dont le prix au m² **est compris entre 2 000 € et 7 000 €**

### 🧠 Pourquoi ce choix ?
Ce filtrage permet :
- De réduire l’impact des valeurs aberrantes ou mal saisies dans le fichier DVF (ex : erreurs humaines, ventes atypiques),
- De renforcer la stabilité des modèles de régression, qui sont sensibles aux grandes variations de valeurs cibles,
- D’obtenir des prédictions plus réalistes sur la majorité des biens du marché.

In [ ]:
# Pour les maisons
df_maisons = df_maison_clean[(df_maison_clean["prix_m2"] >= 1500) & (df_maison_clean["prix_m2"] <= 5000)]

# # Pour les appartements
df_appartements = df_appartement_clean[(df_appartement_clean["prix_m2"] >= 2000) & (df_appartement_clean["prix_m2"] <= 7000)]

df_maisons = df_maison_clean
df_appartements = df_appartement_clean

## 📊 Revisualisation de la distribution du prix au m²

In [ ]:
# Créer la figure et les 2 sous-graphiques côte à côte
fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# --- 1. Histogramme pour les maisons ---
axs[0].hist(df_maisons['prix_m2'], bins=30, color='orange', edgecolor='w')
axs[0].axvline(np.median(df_maisons['prix_m2']), color='red', linestyle='--', label='Médiane')
axs[0].set_title("Maisons - prix_m2")
axs[0].set_xlabel("prix_m2")
axs[0].set_ylabel("Nombre de biens")
axs[0].grid(True)
axs[0].legend()

# --- 2. Histogramme pour les appartements ---
axs[1].hist(df_appartements['prix_m2'], bins=30, color='orange', edgecolor='w')
axs[1].axvline(np.median(df_appartements['prix_m2']), color='red', linestyle='--', label='Médiane')
axs[1].set_title("Appartements - prix_m2")
axs[1].set_xlabel("prix_m2")
axs[1].grid(True)
axs[1].legend()

# Ajuster l'espacement
plt.tight_layout()
plt.show()

## 📈 Analyse de la corrélation entre variables
Cette section permet d’étudier les relations linéaires entre les différentes caractéristiques des biens immobiliers et le prix au m² (prix_m2), séparément pour les maisons et les appartements.

### 🧠 Intérêt de la corrélation :
- Détecter les variables les plus liées au prix_m2, pour guider la sélection de features utiles à la modélisation.
- Comprendre les interdépendances entre les variables (ex : surface terrain et surface bâtie).
- Identifier des relations inutiles ou bruitées, pouvant être écartées dans les futurs modèles.

In [ ]:
# matrices de corrélation
corr_maison = df_maisons[["Nombre de lots",
                        "Surface terrain", "Surface reelle bati", "Nombre pieces principales", "prix_m2"]].corr()
corr_appartement = df_appartements[["Nombre de lots",
                        "Surface terrain", "Surface reelle bati", "Nombre pieces principales", "prix_m2"]].corr()

# Création des sous-figures
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap pour les maisons
sns.heatmap(corr_maison, ax=axs[0], cmap='Oranges', annot=True, fmt=".2f")
axs[0].set_title("Corrélation - Maisons")

# Heatmap pour les appartements
sns.heatmap(corr_appartement, ax=axs[1], cmap='Oranges', annot=True, fmt=".2f")
axs[1].set_title("Corrélation - Appartements")

plt.tight_layout()
plt.show()

## 📊 Séparation des variables (features et cible)

On isole :
- Les variables explicatives (features) : surface, terrain, type de logement, etc.
- La variable cible (`prix_m2`) à prédire.

Cette étape est nécessaire pour entraîner les modèles de machine learning.

In [ ]:
features_maison = ["Surface terrain", "Surface reelle bati"]
features_appartement = ["Nombre de lots", "Surface reelle bati"]

# Données maisons
X_maison = df_maisons[features_maison]
y_maison = df_maisons["prix_m2"]

# Données appartements
X_appartement = df_appartements[features_appartement]
y_appartement = df_appartements["prix_m2"]

## 🧪 Découpage en jeu d'entraînement et de test

Utilisation de `train_test_split()` :
- **80%** des données pour l’entraînement
- **20%** pour le test
- On fixe un `random_state` pour rendre les expériences reproductibles.

Cela permet d’évaluer la capacité du modèle à généraliser à des données jamais vues.

In [ ]:
# Split train/test
X_train_maison, X_test_maison, y_train_maison, y_test_maison = train_test_split(
    X_maison, y_maison, test_size=0.2, random_state=42
)
X_train_appartement, X_test_appartement, y_train_appartement, y_test_appartement = train_test_split(
    X_appartement, y_appartement, test_size=0.2, random_state=42
)

## 🔍 Optimisation des modèles : GridSearchCV
Afin d'obtenir les meilleurs paramètres possibles pour nos modèles de régression, on utilise la méthode GridSearchCV. Cette technique permet de tester automatiquement plusieurs combinaisons d’hyperparamètres pour chaque algorithme, via une validation croisée à 5 plis (cv=5).

### 📌 Objectif :
Maximiser la performance des modèles en minimisant l’erreur quadratique moyenne négative (scoring="neg_mean_squared_error"), ce qui revient à minimiser le MSE.

### 🧪 Hyperparamètres testés

1. DecisionTreeRegressor

```python
param_dt = {
    "max_depth": [3, 5, 10, None],
    "min_samples_split": [2, 5, 10]
}
```
- `max_depth` : profondeur maximale de l’arbre,
- `min_samples_split` : nombre minimal d’échantillons pour un split.

2. RandomForestRegressor

```python
param_rf = {
    "n_estimators": [50, 100],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5]
}
```
- `n_estimators` : nombre d’arbres dans la forêt,
- `max_depth` et `min_samples_split` : mêmes rôles que pour l’arbre.

3. XGBRegressor
```python
param_xgb = {
    "n_estimators": [50, 100],
    "max_depth": [3, 5, 10],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.8, 1],
}
```
- `learning_rate` : taille des pas d’optimisation (influence sur la vitesse/apprentissage),
- `subsample` : proportion d’échantillons utilisés pour chaque arbre (pour éviter l’overfitting).

### ⚙️ Fonction optimize_models()
Cette fonction effectue les étapes suivantes pour chaque algorithme :

1. Lance un GridSearchCV avec les hyperparamètres spécifiés,
2. Entraîne le modèle sur les données d’entraînement (X_train, y_train),
3. Sélectionne le meilleur modèle trouvé (grâce à .best_estimator_),
4. Affiche les meilleurs paramètres retenus,
5. Retourne les trois modèles optimisés.

### 🏠 Double entraînement : maison vs appartement
La fonction est appelée deux fois :

- Une fois sur les données des maisons (X_train_maison, y_train_maison),
- Une autre sur celles des appartements (X_train_appartement, y_train_appartement).

Cela permet de tenir compte des spécificités de chaque type de bien, et d’avoir des modèles spécialisés pour prédire le prix au m².

In [ ]:
# Paramètres pour GridSearch
param_dt = {
    "max_depth": [3, 5, 10, None],
    "min_samples_split": [2, 5, 10]
}

param_rf = {
    "n_estimators": [50, 100],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5]
}

param_xgb = {
    "n_estimators": [50, 100],
    "max_depth": [3, 5, 10],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.8, 1],
}

def optimize_models(X_train, y_train):
    # DecisionTree
    grid_dt = GridSearchCV(
        DecisionTreeRegressor(random_state=42),
        param_dt,
        cv=5,
        scoring="neg_mean_squared_error",
        n_jobs=-1,
    )
    grid_dt.fit(X_train, y_train)
    best_dt = grid_dt.best_estimator_
    print(f"Best Decision Tree params: {grid_dt.best_params_}")

    # RandomForest
    grid_rf = GridSearchCV(
        RandomForestRegressor(random_state=42),
        param_rf,
        cv=5,
        scoring="neg_mean_squared_error",
        n_jobs=-1,
    )
    grid_rf.fit(X_train, y_train)
    best_rf = grid_rf.best_estimator_
    print(f"Best Random Forest params: {grid_rf.best_params_}")
    
    # XGBoost
    grid_xgb = GridSearchCV(
        XGBRegressor(random_state=42, tree_method='hist'),
        param_xgb,
        cv=5,
        scoring="neg_mean_squared_error",
        n_jobs=-1,
        verbose=0
    )
    grid_xgb.fit(X_train, y_train)
    best_xgb = grid_xgb.best_estimator_
    print(f"Best XGBoost params: {grid_xgb.best_params_}")

    return best_dt, best_rf, best_xgb

# Exemple d’appel
best_dt_maison, best_rf_maison, best_xgb_maison = optimize_models(X_train_maison, y_train_maison)
best_dt_appartement, best_rf_appartement, best_xgb_appartement = optimize_models(X_train_appartement, y_train_appartement)


## 🤖 Entraînement de plusieurs modèles de régression

On compare les performances de plusieurs algorithmes :
- `LinearRegression` : modèle linéaire simple
- `DecisionTreeRegressor` : arbre de décision
- `RandomForestRegressor` : ensemble d'arbres (bagging)
- `XGBRegressor` : boosting par gradient
- `VotingRegressor` : combine les 4 modèles précédents

L’objectif est d’identifier celui qui prédit le mieux le prix au m².

In [ ]:
# VotingRegressor enrichi
voting_maison = VotingRegressor(
    estimators=[
        ("lr", LinearRegression()),
        ("dt", best_dt_maison),
        ("rf", best_rf_maison),
        ("xgb", best_xgb_maison)
    ]
)

voting_appartement = VotingRegressor(
    estimators=[
        ("lr", LinearRegression()),
        ("dt", best_dt_appartement),
        ("rf", best_rf_appartement),
        ("xgb", best_xgb_appartement)
    ]
)

# Pipelines
pipeline_maison = Pipeline([
    ("scaler", StandardScaler()),
    ("voting", voting_maison),
])

pipeline_appartement = Pipeline([
    ("scaler", StandardScaler()),
    ("voting", voting_appartement),
])

pipeline_maison.fit(X_train_maison, y_train_maison)
pipeline_appartement.fit(X_train_appartement, y_train_appartement)

## 📈 Évaluation des modèles sur le jeu de test

Pour chaque modèle, on calcule :
- `MSE` : l'erreur quadratique moyenne
- `RMSE` : la racine de l'erreur quadratique moyenne
- `MAE` : l'erreur absolue moyenne
- `R2 score`: la proportion de la variance de la variable cible

Cela nous permet de comparer leur précision et robustesse.


In [ ]:
def compare_models(X_train, X_test, y_train, y_test, bien_label, best_dt, best_rf, best_xgb, pipeline_votingRegressor):
    models = {
        "LinearRegression": LinearRegression(),
        "DecisionTree": best_dt,
        "RandomForest": best_rf,
        "XGBoost": best_xgb,
        "VotingRegressor": pipeline_votingRegressor
    }

    pipeline_export = {}
    results = []
    for name, model in models.items():
        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("model", model)
        ])
        pipe.fit(X_train, y_train)
        pipeline_export[name] = pipe
        y_pred = pipe.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        results.append({
            "Modele": name,
            "MSE": mse,
            "RMSE": rmse,
            "MAE": mae,
            "R2": r2
        })

    df_result = pd.DataFrame(results).sort_values(by="MSE")
    print(f"\n📋 Résultats comparatifs - {bien_label}")
    print(df_result.to_string(index=False))
    return results, pipeline_export, df_result

# Comparaison pour maisons et appartements
results_maisons, pipeline_modele_maison, dt_classement_maison = compare_models(
    X_train_maison, X_test_maison, y_train_maison, y_test_maison,
    "MAISON", best_dt_maison, best_rf_maison, best_xgb_maison, pipeline_maison
)

results_appartements, pipeline_modele_appartement, dt_classement_appartement = compare_models(
    X_train_appartement, X_test_appartement, y_train_appartement, y_test_appartement,
    "APPARTEMENT", best_dt_appartement, best_rf_appartement, best_xgb_appartement, pipeline_appartement
)

## 💾 Sauvegarde des meilleurs modèles

On sauvegarde le meilleur modèle via `joblib.dump()` pour le réutiliser dans l'API FastAPI.

Deux modèles sont sauvegardés :
- Un pour les maisons
- Un pour les appartements

In [ ]:
import json

result = {
    "maison": results_maisons,
    "appartement": results_appartements
}

# #Sauvegarde des données dans un fichier JSON
with open("resultats_modeles_maison_appartement.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

# 💾 Sauvegarde des pipelines
joblib.dump(pipeline_modele_maison, "../models/pipeline_maison_models.pkl")
joblib.dump(pipeline_modele_appartement, "../models/pipeline_appartement_models.pkl")

dt_classement_maison

meilleur_modele_maison = dt_classement_maison.iloc[0]["Modele"]
meilleur_modele_appartement = dt_classement_appartement.iloc[0]["Modele"]

# 💾 Sauvegarde des modeles pour Lille
os.makedirs("../app/models/Lille", exist_ok=True)
joblib.dump(pipeline_modele_maison[meilleur_modele_maison], "../app/models/Lille/models_maison_Lille.pkl")
joblib.dump(pipeline_modele_appartement[meilleur_modele_appartement], "../app/models/Lille/models_appartement_Lille.pkl")